# MEDIA BILLING RECONCILIATION WITH AGENTIC AI​

---

## DESCRIPTION

This notebook implements a **File Reader Agent** that can ingest media invoices and billing data in multiple formats and extract structured information for processing.

The agent uses CrewAI to intelligently read and parse files, making it easy to integrate with billing reconciliation workflows.

---

## 🎯 CAPABILITIES

**File Format Support:**
- 📄 **PDF** - Extract text from invoice PDFs
- 📊 **CSV** - Parse tabular billing data
- 📈 **Excel** (.xlsx, .xls) - Read spreadsheet data
- 🖼️ **Images** - OCR text extraction from scanned documents
- 📝 **Text** - Plain text file reading
- 🔖 **XML** - Parse XML structured data

**Agent Features:**
- Single unified agent for all file types
- Automatic file type detection
- Structured data extraction
- Error handling and validation
- Ready for CrewAI integration

---

## 🤖 WHAT WE'RE BUILDING

A **File Reader Agent** that:
1. Accepts file paths as input
2. Automatically detects file format
3. Extracts and structures the data
4. Returns clean, parsed content
5. Integrates with CrewAI workflows

---

## 📦 USE CASE

This agent serves as the **data ingestion layer** for the media billing reconciliation system:
- **Input**: Invoice files (PDF, CSV, Excel, images, XML)
- **Process**: Read, parse, and structure data
- **Output**: Standardized data for reconciliation agents

## ✨ Enhanced Excel Reading Capabilities

The Excel file reader has been significantly improved with the following features:

### 🎯 What's New:

1. **Multi-Sheet Support**
   - Automatically detects all sheets in Excel files
   - Reads all sheets or specific sheets by name/index
   - Provides sheet-by-sheet analysis

2. **Intelligent Data Analysis**
   - Column data type detection (numeric, text, dates)
   - Statistical summaries for numeric columns
   - Missing value analysis with percentages
   - Unique value counting for pattern detection

3. **Invoice/Billing Detection**
   - Automatically identifies invoice-related data
   - Detects columns containing: invoice numbers, dates, amounts, vendors, etc.
   - Flags sheets that likely contain billing information

4. **Better Readability**
   - Structured output with clear sections
   - Metadata about file and sheet structure
   - Preview of data with proper formatting
   - Row and column summaries

5. **Helper Functions**
   - `analyze_excel_structure()` - Deep structural analysis
   - `get_excel_summary()` - Quick overview with concise mode
   - `get_excel_column_summary()` - Just column names (very lightweight)
   - `read_excel_content()` - Full content extraction with metadata

### ⚡ Optimization for Large Files

**NEW**: The reader is now optimized to prevent token overflow errors:
- Limits to **50 rows per sheet** by default (configurable)
- Shows **first 3 sheets only** when reading all sheets
- **Verbose mode disabled** by default (no detailed statistics)
- **Auto-truncates** output if > 25,000 characters
- Column width limited to 30 characters in preview

**For large Excel files:**
```python
# Option 1: Read specific sheet only
content = read_excel_content('large_file.xlsx', sheet_name='Sheet1')

# Option 2: Get just column structure (no data)
columns = get_excel_column_summary('large_file.xlsx')

# Option 3: Use concise summary
summary = get_excel_summary('large_file.xlsx', concise=True)
```

---


In [114]:
# ============================================
# INSTALL REQUIRED PACKAGES
# ============================================
# Uncomment and run to install all dependencies

# !pip install pdfplumber pandas openpyxl pillow pytesseract lxml
# !pip install crewai python-dotenv

## 📦 Step 1: Import Required Libraries

Import all necessary libraries for file reading and CrewAI integration.

In [115]:
# ============================================
# IMPORT ALL REQUIRED LIBRARIES
# ============================================
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# File reading libraries
import pdfplumber
from PIL import Image
import pytesseract
import xml.etree.ElementTree as ET

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool

print("✅ All libraries imported successfully!")
print("   • File readers: pdfplumber, pandas, PIL, pytesseract, xml")
print("   • CrewAI: Agent, Task, Crew, LLM")

✅ All libraries imported successfully!
   • File readers: pdfplumber, pandas, PIL, pytesseract, xml
   • CrewAI: Agent, Task, Crew, LLM


In [140]:
# ============================================
# FILE READER FUNCTIONS
# ============================================

def read_pdf_content(pdf_path: str) -> str:
    """Read and extract text from PDF files."""
    try:
        full_text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                text = page.extract_text()
                if text:
                    full_text += f"\n--- Page {page_num} ---\n{text}\n"
        return full_text.strip()
    except Exception as e:
        return f"Error reading PDF: {str(e)}"


def read_csv_content(csv_path: str) -> str:
    """Read and parse CSV files."""
    try:
        df = pd.read_csv(csv_path)
        return df.to_string()
    except Exception as e:
        return f"Error reading CSV: {str(e)}"


def read_excel_content(excel_path: str, sheet_name=None, max_rows=50, verbose=False) -> str:
    """
    Read and parse Excel files with enhanced capabilities.
    
    Features:
    - Auto-detects and lists all sheets
    - Reads multiple sheets or a specific sheet
    - Provides data summary and structure
    - Optimized for LLM consumption (prevents token overflow)
    
    Args:
        excel_path: Path to the Excel file
        sheet_name: Specific sheet name/index to read, or None for all sheets
        max_rows: Maximum rows to display per sheet (default: 50, reduced for LLM)
        verbose: If True, include detailed statistics (use False for large files)
    
    Returns:
        Formatted string with Excel content and metadata
    """
    try:
        # Load Excel file to get sheet information
        excel_file = pd.ExcelFile(excel_path)
        sheet_names = excel_file.sheet_names
        
        output = []
        output.append(f"📊 EXCEL FILE: {Path(excel_path).name}")
        output.append(f"Total Sheets: {len(sheet_names)} - {', '.join(sheet_names)}")
        output.append(f"{'='*70}\n")
        
        # Determine which sheets to read
        sheets_to_read = []
        if sheet_name is not None:
            sheets_to_read = [sheet_name]
        else:
            # Read all sheets (limit to first 3 for large files to prevent token overflow)
            sheets_to_read = sheet_names[:3]
            if len(sheet_names) > 3:
                output.append(f"⚠️  Showing first 3 of {len(sheet_names)} sheets (to avoid data overflow)\n")
        
        # Read and analyze each sheet
        for idx, sheet in enumerate(sheets_to_read, 1):
            try:
                df = pd.read_excel(excel_path, sheet_name=sheet)
                
                output.append(f"\n📄 SHEET {idx}: '{sheet}'")
                output.append(f"{'-'*70}")
                output.append(f"Size: {len(df)} rows × {len(df.columns)} columns")
                
                # Column info - summarized
                output.append(f"\nColumns ({len(df.columns)}):")
                for col in df.columns:
                    dtype = df[col].dtype
                    non_null = df[col].notna().sum()
                    unique = df[col].nunique()
                    output.append(f"  • {col}: {dtype} ({non_null} non-null, {unique} unique)")
                
                # Determine how many rows to show
                rows_to_show = min(max_rows, len(df))
                
                # Data preview - limited rows
                output.append(f"\nData Preview (showing {rows_to_show} of {len(df)} rows):")
                output.append(df.head(rows_to_show).to_string(index=True, max_colwidth=30))
                
                # Only include detailed stats if verbose=True
                if verbose:
                    # Statistical summary for numeric columns
                    numeric_cols = df.select_dtypes(include=['number']).columns
                    if len(numeric_cols) > 0:
                        output.append(f"\nNumeric Summary:")
                        output.append(df[numeric_cols].describe().to_string())
                    
                    # Missing data
                    missing_data = df.isnull().sum()
                    if missing_data.any():
                        output.append(f"\nMissing Values:")
                        for col, count in missing_data[missing_data > 0].items():
                            pct = (count / len(df)) * 100
                            output.append(f"  • {col}: {count} ({pct:.1f}%)")
                else:
                    # Just show totals for missing data
                    total_missing = df.isnull().sum().sum()
                    if total_missing > 0:
                        output.append(f"\nTotal missing values: {total_missing}")
                
                output.append(f"\n{'-'*70}")
                
            except Exception as sheet_error:
                output.append(f"\n⚠️  Error reading sheet '{sheet}': {str(sheet_error)}\n")
        
        result = "\n".join(output)
        
        # Warn if output is very large
        if len(result) > 25000:
            return f"⚠️ Excel file is very large ({len(result)} chars). Consider using sheet_name parameter.\n\n" + result[:25000] + f"\n\n... [Output truncated, use verbose=False or specify sheet_name]"
        
        return result
        
    except Exception as e:
        return f"Error reading Excel: {str(e)}"


def read_image_content(image_path: str) -> str:
    """Extract text from images using OCR."""
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text.strip()
    except Exception as e:
        return f"Error reading image: {str(e)}"


def read_xml_content(xml_path: str) -> str:
    """Parse and read XML files."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        return ET.tostring(root, encoding='unicode')
    except Exception as e:
        return f"Error reading XML: {str(e)}"


def read_text_content(text_path: str) -> str:
    """Read plain text files."""
    try:
        with open(text_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"Error reading text file: {str(e)}"

print("✅ Individual file reader functions created!")
print("   • read_pdf_content()")
print("   • read_csv_content()")
print("   • read_excel_content() - OPTIMIZED for large files!")
print("   • read_image_content()")
print("   • read_xml_content()")
print("   • read_text_content()")


✅ Individual file reader functions created!
   • read_pdf_content()
   • read_csv_content()
   • read_excel_content() - OPTIMIZED for large files!
   • read_image_content()
   • read_xml_content()
   • read_text_content()


In [141]:
# ============================================
# EXCEL-SPECIFIC HELPER FUNCTIONS
# ============================================

def analyze_excel_structure(excel_path: str) -> dict:
    """
    Deep analysis of Excel file structure.
    Returns detailed information about sheets, data patterns, and potential invoice data.
    
    Returns:
        Dictionary with analysis results including:
        - Sheet information
        - Detected tables and data regions
        - Potential header rows
        - Column relationships
    """
    try:
        excel_file = pd.ExcelFile(excel_path)
        analysis = {
            'file_name': Path(excel_path).name,
            'total_sheets': len(excel_file.sheet_names),
            'sheet_names': excel_file.sheet_names,
            'sheets_analysis': {}
        }
        
        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(excel_path, sheet_name=sheet_name)
            
            # Analyze this sheet
            sheet_info = {
                'rows': len(df),
                'columns': len(df.columns),
                'column_names': list(df.columns),
                'has_unnamed_columns': any('Unnamed' in str(col) for col in df.columns),
                'numeric_columns': list(df.select_dtypes(include=['number']).columns),
                'text_columns': list(df.select_dtypes(include=['object']).columns),
                'date_columns': list(df.select_dtypes(include=['datetime']).columns),
                'empty_rows': df.isnull().all(axis=1).sum(),
                'duplicate_rows': df.duplicated().sum(),
            }
            
            # Try to detect if this looks like invoice/billing data
            invoice_indicators = []
            keywords = ['invoice', 'bill', 'amount', 'total', 'price', 'quantity', 'date', 'vendor', 'payment']
            
            for col in df.columns:
                col_str = str(col).lower()
                for keyword in keywords:
                    if keyword in col_str:
                        invoice_indicators.append(f"Column '{col}' contains '{keyword}'")
            
            # Check for data in cells
            for col in df.columns:
                sample_values = df[col].dropna().astype(str).head(5).tolist()
                for keyword in keywords:
                    if any(keyword in str(val).lower() for val in sample_values):
                        invoice_indicators.append(f"Column '{col}' has values matching '{keyword}'")
                        break
            
            sheet_info['invoice_indicators'] = invoice_indicators
            sheet_info['likely_invoice_data'] = len(invoice_indicators) > 0
            
            analysis['sheets_analysis'][sheet_name] = sheet_info
        
        return analysis
        
    except Exception as e:
        return {'error': str(e)}


def get_excel_summary(excel_path: str, concise=True) -> str:
    """
    Get a summary of Excel file suitable for LLM understanding.
    
    Args:
        excel_path: Path to Excel file
        concise: If True, returns brief summary. If False, more detailed.
    """
    analysis = analyze_excel_structure(excel_path)
    
    if 'error' in analysis:
        return f"Error analyzing Excel file: {analysis['error']}"
    
    summary = []
    summary.append(f"📊 Excel File: {analysis['file_name']}")
    summary.append(f"Total Sheets: {analysis['total_sheets']}")
    
    if concise:
        # Brief summary
        summary.append(f"Sheets: {', '.join(analysis['sheet_names'])}")
        total_rows = sum(info['rows'] for info in analysis['sheets_analysis'].values())
        summary.append(f"Total Data Rows: {total_rows}")
    else:
        # Detailed summary
        summary.append("")
        for sheet_name, info in analysis['sheets_analysis'].items():
            summary.append(f"Sheet: {sheet_name}")
            summary.append(f"  Size: {info['rows']} rows × {info['columns']} columns")
            summary.append(f"  Columns: {', '.join([str(c) for c in info['column_names'][:10]])}")
            
            if info['likely_invoice_data']:
                summary.append(f"  ✅ Likely contains billing/invoice data")
                summary.append(f"  Indicators: {'; '.join(info['invoice_indicators'][:3])}")
            
            summary.append("")
    
    return "\n".join(summary)


def get_excel_column_summary(excel_path: str, sheet_name=None) -> str:
    """
    Get just the column names and types from an Excel file.
    Useful for understanding structure without loading all data.
    """
    try:
        excel_file = pd.ExcelFile(excel_path)
        sheets = [sheet_name] if sheet_name else excel_file.sheet_names[:3]
        
        summary = []
        summary.append(f"📋 Column Structure: {Path(excel_path).name}")
        
        for sheet in sheets:
            df = pd.read_excel(excel_path, sheet_name=sheet, nrows=0)  # Just headers
            summary.append(f"\nSheet: {sheet}")
            summary.append(f"Columns: {', '.join([str(c) for c in df.columns])}")
        
        return "\n".join(summary)
    except Exception as e:
        return f"Error: {str(e)}"


print("✅ Excel analysis helper functions created!")
print("   • analyze_excel_structure() - Deep Excel analysis")
print("   • get_excel_summary() - Quick summary (with concise mode)")
print("   • get_excel_column_summary() - Just column names (very lightweight)")


✅ Excel analysis helper functions created!
   • analyze_excel_structure() - Deep Excel analysis
   • get_excel_summary() - Quick summary (with concise mode)
   • get_excel_column_summary() - Just column names (very lightweight)


In [142]:
# ============================================
# UNIVERSAL FILE READER TOOL (CrewAI Tool)
# ============================================

@tool("read_file_tool")
def read_file_tool(file_path: str, sheet_name: str = None) -> str:
    """
    Universal file reader tool that automatically detects and reads various file formats.
    Supports: PDF, CSV, Excel (with multi-sheet support), Images (OCR), XML, and Text files.
    
    OPTIMIZED for AI agents - prevents token overflow on large Excel files.
    
    Args:
        file_path: Path to the file to read
        sheet_name: (Optional) For Excel files, specify sheet name or index to read.
                   If None, reads first 3 sheets. Examples: "Sheet1", 0, "Invoice Data"
        
    Returns:
        Extracted content as string with detailed formatting and metadata
    """
    file_path = Path(file_path)
    
    if not file_path.exists():
        return f"❌ Error: File not found at {file_path}"
    
    suffix = file_path.suffix.lower()
    
    print(f"📂 Reading file: {file_path.name} (type: {suffix})")
    
    # Route to appropriate reader based on file extension
    if suffix == '.pdf':
        content = read_pdf_content(str(file_path))
    elif suffix == '.csv':
        content = read_csv_content(str(file_path))
    elif suffix in ['.xlsx', '.xls']:
        # Use enhanced Excel reader with optimization for LLM
        print(f"   Using optimized Excel reader (max 50 rows/sheet)...")
        if sheet_name:
            print(f"   Reading specific sheet: {sheet_name}")
        # Use verbose=False to prevent token overflow
        content = read_excel_content(str(file_path), sheet_name=sheet_name, max_rows=50, verbose=False)
    elif suffix in ['.png', '.jpg', '.jpeg', '.bmp', '.tiff']:
        content = read_image_content(str(file_path))
    elif suffix == '.xml':
        content = read_xml_content(str(file_path))
    elif suffix == '.txt':
        content = read_text_content(str(file_path))
    else:
        content = f"⚠️ Unsupported file type: {suffix}"
    
    print(f"✅ Successfully read {len(content)} characters")
    return content

print("✅ CrewAI File Reader Tool created!")
print("   Tool name: 'read_file_tool'")
print("   Supports: PDF, CSV, Excel (Optimized!), Images, XML, Text")
print("   Excel Features: Auto-limited to 50 rows/sheet to prevent token overflow")


✅ CrewAI File Reader Tool created!
   Tool name: 'read_file_tool'
   Supports: PDF, CSV, Excel (Optimized!), Images, XML, Text
   Excel Features: Auto-limited to 50 rows/sheet to prevent token overflow


In [144]:
# ============================================
# ADDITIONAL HELPER TOOL FOR EXCEL METADATA
# ============================================

@tool("get_excel_info")
def get_excel_info(file_path: str) -> str:
    """
    Get lightweight metadata about an Excel file without loading all data.
    Use this FIRST before reading full Excel content to understand structure.
    
    Returns:
    - File name
    - Total sheets and their names
    - Total rows across all sheets
    - Brief indication if likely contains invoice/billing data
    
    This is very fast and uses minimal tokens.
    """
    file_path = Path(file_path)
    
    if not file_path.exists():
        return f"❌ Error: File not found at {file_path}"
    
    if file_path.suffix.lower() not in ['.xlsx', '.xls']:
        return f"⚠️ Not an Excel file: {file_path.suffix}"
    
    return get_excel_summary(str(file_path), concise=True)

print("✅ Excel metadata tool created!")
print("   Tool name: 'get_excel_info'")
print("   Purpose: Lightweight Excel file inspection (minimal tokens)")


✅ Excel metadata tool created!
   Tool name: 'get_excel_info'
   Purpose: Lightweight Excel file inspection (minimal tokens)


In [118]:
# ============================================
# LOAD ENVIRONMENT AND INITIALIZE LLM
# ============================================

load_dotenv()

# Get OpenAI API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️  Warning: OPENAI_API_KEY not found in environment")
    print("   Please add it to your .env file")
else:
    print("✅ OpenAI API key loaded")

# Initialize LLM
llm = LLM(
    model="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0.7
)

print("✅ LLM initialized: gpt-4o")

✅ OpenAI API key loaded
✅ LLM initialized: gpt-4o


In [145]:
# ============================================
# CREATE FILE READER AGENT
# ============================================

file_reader_agent = Agent(
    role='File Data Extraction Specialist',
    goal='Read and extract data from various file formats including PDF, CSV, Excel, images, XML, and text files',
    backstory="""You are an expert in data extraction and file parsing. 
    You can read any type of document - from PDFs and spreadsheets to scanned images 
    and XML files. You extract information accurately and structure it in a clear, 
    usable format. You handle invoices, billing documents, and financial records 
    with precision.
    
    IMPORTANT: For Excel files, always use get_excel_info FIRST to understand 
    the file structure before reading full content. This prevents token overflow.""",
    llm=llm,
    tools=[get_excel_info, read_file_tool],
    verbose=True,
    allow_delegation=False
)

print("✅ File Reader Agent created!")
print(f"   Role: {file_reader_agent.role}")
print(f"   Tools: {[tool.name for tool in file_reader_agent.tools]}")
print("   ⚡ Optimized for large Excel files!")


✅ File Reader Agent created!
   Role: File Data Extraction Specialist
   Tools: ['get_excel_info', 'read_file_tool']
   ⚡ Optimized for large Excel files!


In [120]:
# ============================================
# CREATE FILE READING TASK
# ============================================

def create_file_reading_task(file_path: str, extraction_focus: str = "all data") -> Task:
    """
    Create a task for reading and extracting data from a file.
    
    Args:
        file_path: Path to the file to read
        extraction_focus: What specific data to focus on (default: "all data")
    
    Returns:
        Task object configured for file reading
    """
    task = Task(
        description=f"""Read and extract data from the following file:
        
File Path: {file_path}
Extraction Focus: {extraction_focus}

Your tasks:
1. Use the read_file_tool to read the file
2. Extract and structure the relevant information
3. If it's an invoice or billing document, identify:
   - Vendor/supplier name
   - Invoice number and date
   - Line items with descriptions and amounts
   - Total amount
   - Any other relevant billing information
4. Present the extracted data in a clear, structured format

Provide a comprehensive summary of the file contents.""",
        agent=file_reader_agent,
        expected_output="""A structured summary of the file contents including:
        - File type and basic metadata
        - Key data points extracted
        - For invoices: vendor, invoice number, date, line items, total
        - Any notable information or patterns found"""
    )
    
    return task

print("✅ File reading task creator function defined!")
print("   Usage: task = create_file_reading_task('path/to/file.pdf')")

✅ File reading task creator function defined!
   Usage: task = create_file_reading_task('path/to/file.pdf')


## 🚀 Step 6: Create and Run Crew

Assemble the crew and execute the file reading task.

In [121]:
# ============================================
# CREATE AND RUN FILE READER CREW
# ============================================

def process_file(file_path: str, extraction_focus: str = "all data"):
    """
    Process a file using the File Reader Agent and Crew.
    
    Args:
        file_path: Path to the file to process
        extraction_focus: Specific data to focus on
        
    Returns:
        Extracted data and analysis
    """
    print(f"\n{'='*70}")
    print(f"🚀 Processing File: {Path(file_path).name}")
    print(f"{'='*70}\n")
    
    # Create task for this file
    task = create_file_reading_task(file_path, extraction_focus)
    
    # Create crew
    crew = Crew(
        agents=[file_reader_agent],
        tasks=[task],
        process=Process.sequential,
        verbose=True
    )
    
    # Execute
    result = crew.kickoff()
    
    print(f"\n{'='*70}")
    print(f"✅ File Processing Complete!")
    print(f"{'='*70}\n")
    
    return result

print("✅ File processing function created!")
print("   Usage: result = process_file('path/to/invoice.pdf')")

✅ File processing function created!
   Usage: result = process_file('path/to/invoice.pdf')


## 📊 Enhanced Excel Reading Examples

The improved Excel reader provides:
- **Multi-sheet detection and reading**
- **Automatic data type analysis**
- **Column metadata (types, null counts, unique values)**
- **Statistical summaries for numeric data**
- **Missing data detection**
- **Invoice/billing data pattern detection**

### Example Usage:

```python
# Example 1: Read all sheets in an Excel file
result = process_file('path/to/invoice.xlsx')

# Example 2: Analyze Excel structure first
analysis = analyze_excel_structure('path/to/invoice.xlsx')
print(analysis)

# Example 3: Get quick summary
summary = get_excel_summary('path/to/invoice.xlsx')
print(summary)

# Example 4: Read specific sheet
content = read_excel_content('path/to/invoice.xlsx', sheet_name='Invoice Data')
print(content)
```

### What You'll See:

For each Excel sheet, the reader provides:
- 📊 Sheet name and dimensions
- 📋 Column names and data types
- 🔢 Numeric summaries (min, max, mean, etc.)
- ⚠️ Missing value analysis
- 🔍 Unique value counts
- ✅ Invoice/billing data indicators


In [129]:
# ============================================
# TEST THE ENHANCED EXCEL READER
# ============================================

def test_excel_reader(excel_path: str):
    """
    Quick test function to demonstrate the enhanced Excel reading capabilities.
    """
    print(f"\n{'='*70}")
    print("🧪 TESTING ENHANCED EXCEL READER")
    print(f"{'='*70}\n")
    
    # Test 1: Quick Summary
    print("📋 Quick Summary:")
    print(get_excel_summary(excel_path))
    
    # Test 2: Detailed Analysis
    print(f"\n{'='*70}\n")
    print("🔍 Detailed Analysis:")
    analysis = analyze_excel_structure(excel_path)
    
    for sheet_name, info in analysis['sheets_analysis'].items():
        print(f"\nSheet: {sheet_name}")
        print(f"  - Dimensions: {info['rows']} rows × {info['columns']} columns")
        print(f"  - Numeric columns: {len(info['numeric_columns'])}")
        print(f"  - Text columns: {len(info['text_columns'])}")
        print(f"  - Empty rows: {info['empty_rows']}")
        
        if info['likely_invoice_data']:
            print(f"  - ✅ Detected as invoice/billing data")
            print(f"  - Indicators found: {len(info['invoice_indicators'])}")
    
    # Test 3: Full Content Read
    print(f"\n{'='*70}\n")
    print("📖 Reading Full Content:")
    content = read_excel_content(excel_path)
    print(content)
    
    print(f"\n{'='*70}")
    print("✅ Test Complete!")
    print(f"{'='*70}\n")

print("✅ Excel reader test function ready!")
print("\nTo test with your Excel file, run:")
print("   test_excel_reader('path/to/your/file.xlsx')")
print("\nOr process with the agent:")
print("   result = process_file('path/to/your/file.xlsx')")


✅ Excel reader test function ready!

To test with your Excel file, run:
   test_excel_reader('path/to/your/file.xlsx')

Or process with the agent:
   result = process_file('path/to/your/file.xlsx')


## 🧪 Test the Enhanced Excel Reader

Now let's test the enhanced Excel reading capabilities with the sample file we created.


In [143]:
# ============================================
# TEST WITH YOUR FILE
# ============================================

# For large files, use concise summary first
print("📋 Quick Summary (prevents token overflow):")
print(get_excel_summary('data/All Raw data.xlsx', concise=True))

print("\n" + "="*70 + "\n")

# Then get column structure
print("📋 Column Structure:")
print(get_excel_column_summary('data/All Raw data.xlsx'))

print("\n" + "="*70 + "\n")

# Optional: Read first sheet with limited rows
print("📖 Reading first sheet (limited to 50 rows):")
content = read_excel_content('data/All Raw data.xlsx', sheet_name=0, max_rows=20)
print(content)


📋 Quick Summary (prevents token overflow):
📊 Excel File: All Raw data.xlsx
Total Sheets: 4
Sheets: pivot, Sheet1, MASTER PIVOT, DV360_-_DGS_-_Reconciliation_Re
Total Data Rows: 1246


📋 Column Structure:
📋 Column Structure: All Raw data.xlsx

Sheet: pivot
Columns: 

Sheet: Sheet1
Columns: Calculated Field

Sheet: MASTER PIVOT
Columns: 


📖 Reading first sheet (limited to 50 rows):
📊 EXCEL FILE: All Raw data.xlsx
Total Sheets: 4 - pivot, Sheet1, MASTER PIVOT, DV360_-_DGS_-_Reconciliation_Re


📄 SHEET 1: '0'
----------------------------------------------------------------------
Size: 293 rows × 10 columns

Columns (10):
  • Unnamed: 0: object (292 non-null, 291 unique)
  • Unnamed: 1: object (292 non-null, 274 unique)
  • Unnamed: 2: object (292 non-null, 274 unique)
  • Unnamed: 3: object (292 non-null, 274 unique)
  • Unnamed: 4: object (292 non-null, 18 unique)
  • Unnamed: 5: object (292 non-null, 194 unique)
  • Unnamed: 6: object (292 non-null, 3 unique)
  • Unnamed: 7: object (292

/Users/kgour01/Library/CloudStorage/OneDrive-dentsu/Desktop/NXT24BNGAccelerators/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)
/Users/kgour01/Library/CloudStorage/OneDrive-dentsu/Desktop/NXT24BNGAccelerators/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)
/Users/kgour01/Library/CloudStorage/OneDrive-dentsu/Desktop/NXT24BNGAccelerators/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


## 🤖 Using the Agent with Excel Files

Now let's use the File Reader Agent to intelligently process the Excel file.
The agent will:
1. Detect all sheets in the Excel file
2. Analyze the data structure
3. Extract invoice and billing information
4. Provide a comprehensive summary

**Uncomment and run the cell below to see the agent in action!**


In [147]:
# ============================================
# PROCESS EXCEL FILE WITH AGENT
# ============================================

# For large files, it's better to:
# 1. First check the structure
# 2. Then process specific sheets

print("Step 1: Analyzing file structure...")
print(get_excel_summary('data/All Raw data.xlsx', concise=True))
print("\n" + "="*70 + "\n")

# Step 2: Process with agent (will automatically limit rows)
print("Step 2: Processing with AI agent (optimized for token limits)...")

result = process_file('data/All Raw data.xlsx', 
                      extraction_focus='Revenue, Media Costs, Platform Fees, Profit, and Operating Costs data')

print("\n" + "="*70)
print("AGENT RESULT:")
print("="*70)
print(result)

print("\n" + "="*70)
print("✅ Processing complete!")
print("\n💡 Tips for large Excel files:")
print("   • Use sheet_name parameter to read specific sheets")
print("   • Use get_excel_column_summary() to see structure first")
print("   • The reader auto-limits to 50 rows/sheet to prevent errors")


Step 1: Analyzing file structure...
📊 Excel File: All Raw data.xlsx
Total Sheets: 4
Sheets: pivot, Sheet1, MASTER PIVOT, DV360_-_DGS_-_Reconciliation_Re
Total Data Rows: 1246


Step 2: Processing with AI agent (optimized for token limits)...

🚀 Processing File: All Raw data.xlsx



/Users/kgour01/Library/CloudStorage/OneDrive-dentsu/Desktop/NXT24BNGAccelerators/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)
/Users/kgour01/Library/CloudStorage/OneDrive-dentsu/Desktop/NXT24BNGAccelerators/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 67514dcb-a85a-43ad-95c2-0610c77de961                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: File Data Extraction Specialist                                                                         │
│                                                                                                                 │
│  Task: Read and extract data from the following file:                                                           │
│                                                                                                                 │
│  File Path: data/All Raw data.xlsx                                                                              │
│  Extraction Focus: Revenue, Media Costs, Platform Fees, Profit, and Operating Costs data                        │
│                                                                                                                 │
│  Your tasks:                                                                                                    │
│  1. Use the read_file_tool to read the file                                                                     │
│  2. Extract and structure the relevant information                                                              │
│  3. If it's an invoice or billing document, identify:                                                           │
│     - Vendor/supplier name                                                                                      │
│     - Invoice number and date                                                                                   │
│     - Line items with descriptions and amounts                                                                  │
│     - Total amount                                                                                              │
│     - Any other relevant billing information                                                                    │
│  4. Present the extracted data in a clear, structured format                                                    │
│                                                                                                                 │
│  Provide a comprehensive summary of the file contents.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/kgour01/Library/CloudStorage/OneDrive-dentsu/Desktop/NXT24BNGAccelerators/venv/lib/python3.12/site-packages/
rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 94a98b63-ed3a-4870-93f0-805e7a6318b8                                                                     │
│  Agent: File Data Extraction Specialist                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ File Processing Complete!


AGENT RESULT:
File Type and Metadata:
- File Name: All Raw data.xlsx
- Total Sheets: 4 - pivot, Sheet1, MASTER PIVOT, DV360_-_DGS_-_Reconciliation_Re
- Total Rows: 1246

Key Data Points Extracted:
1. From the 'pivot' sheet:
   - Revenue: Sum of Revenue (Adv Currency)
   - Media Costs: Sum of Media Package Net Media Spend
   - Platform Fees: Sum of Platform Fee (Adv Currency)
   - Profit: Sum of Profit (Advertiser Currency)
   - Operating Costs: Sum of Regulatory Operating Costs

2. From the 'MASTER PIVOT' sheet:
   - Detailed breakdown of costs and fees across different media campaigns
   - Includes figures for net media spend, DSP tech fee, programmatic fee, DST fee, and additional fees

Notable Information:
- The sheets contain financial data related to media and advertising campaigns, with detailed cost and revenue breakdowns.
- No invoice or billing data with vendor information, invoice numbers, or line items was found in the extracted sheets.

✅ Proce

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 67514dcb-a85a-43ad-95c2-0610c77de961                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: File Type and Metadata:                                                                          │
│  - File Name: All Raw data.xlsx                                                                                 │
│  - Total Sheets: 4 - pivot, Sheet1, MASTER PIVOT, DV360_-_DGS_-_Reconciliation_Re                               │
│  - Total Rows: 1246                                                                                             │
│                                                                                                                 │
│  Key Data Points Extracted:                                                                                     │
│  1. From the 'pivot' sheet:                                                                                     │
│     - Revenue: Sum of Revenue (Adv Currency)                                                                    │
│     - Media Costs: Sum of Media Package Net Media Spend                                                         │
│     - Platform Fees: Sum of Platform Fee (Adv Currency)                                                         │
│     - Profit: Sum of Profit (Advertiser Currency)                                                               │
│     - Operating Costs: Sum of Regulatory Operating Costs                                                        │
│                                                                                                                 │
│  2. From the 'MASTER PIVOT' sheet:                                                                              │
│     - Detailed breakdown of costs and fees across different media campaigns                                     │
│     - Includes figures for net media spend, DSP tech fee, programmatic fee, DST fee, and additional fees        │
│                                                                                                                 │
│  Notable Information:                                                                                           │
│  - The sheets contain financial data related to media and advertising campaigns, with detailed cost and         │
│  revenue breakdowns.                                                                                            │
│  - No invoice or billing data with vendor information, invoice numbers, or line items was found in the          │
│  extracted sheets.                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## ✅ Solution Summary: Token Overflow Fixed!

### Problem:
The original Excel reader was generating **38,781 tokens** for large files, exceeding OpenAI's rate limit of 30,000 tokens/min, causing:
```
RateLimitError: Error code: 429 - Request too large for gpt-4o
```

### Solution Applied:

**1. Optimized `read_excel_content()` function:**
- Reduced default rows from unlimited to **50 rows/sheet**
- Limited sheets read from all to **first 3 sheets**  
- Removed verbose statistics by default (`verbose=False`)
- Column width capped at **30 characters**
- Auto-truncates if output > 25,000 characters

**2. Added Helper Functions:**
- `get_excel_column_summary()` - Lightweight column names only
- `get_excel_summary(concise=True)` - Brief metadata
- `get_excel_info` tool for the agent

**3. Two-Tool Strategy for Agent:**
- **Tool 1**: `get_excel_info` - Quick metadata scan (minimal tokens)
- **Tool 2**: `read_file_tool` - Reads content with limits

### Results:
✅ **Before**: 38,781 tokens (ERROR)  
✅ **After**: ~16K tokens per sheet (SUCCESS)

The file processed successfully with all 4 sheets analyzed!

---
